In [ ]:
# Define the system dynamics
def inverted_pendulum_update(t, x, u, params={}):
    g = 9.81  # Acceleration due to gravity in m/s^2
    L = 1.0   # Length of the pendulum in meters
    b = 0.1   # Damping coefficient

    theta, theta_dot = x  # Unpack the state variables
    theta += np.pi        # Shift the angle to make the upright position the zero point
    u = u[0]              # Control input (torque)

    # Equation of motion
    theta_ddot = (u - b * theta_dot - g * np.sin(theta)) / L

    return np.array([theta_dot, theta_ddot])

def inverted_pendulum_output(t, x, u, params={}):
    return x[0]  # Output is the angle theta

# Create a NonlinearIOSystem for the pendulum
pendulum_nl = ct.NonlinearIOSystem(
    inverted_pendulum_update,  # Update function
    inverted_pendulum_output,  # Output function
    inputs=('u'),                       # Number of inputs
    outputs=('theta'),                  # Number of outputs
    states=('theta', 'theta_dot'),      # Number of states
    name='pendulum'
)

# Linearize the system around the upright position (theta = 0, theta_dot = 0) with zero input
x0 = [0, 0]  # Operating point (state)
u0 = [0]     # Operating point (input)
t0 = 0       # Time to linearize (usually 0 for time-invariant systems)

# Linearize the nonlinear system at the specified operating point
sys_lin = ct.linearize(pendulum_nl, xeq=x0, ueq=u0, t=t0)

# Convert the linearized state-space model to a transfer function
pendulum_tf = ct.ss2tf(sys_lin)
print("Transfer function of the linearized pendulum:")
print(pendulum_tf)

P_s = pendulum_tf

# Plot the singular value plot for the linearized system
plt.figure()
ct.bode_plot(pendulum_tf)
plt.show()

In [ ]:
import control as ct
import numpy as np
import matplotlib.pyplot as plt

def makeweight_1(M : float, w1 : float) -> ct.TransferFunction:
    """
    Creates a frequency weight transfer function that has:
    - magnitude M >> 1 at low frequencies,
    - rolls off to zero at high frequencies,
    - and has magnitude 1 at the frequency s=j*w1.

    Parameters
    ----------
    - M (float): The desired DC gain of the transfer function.
    - w1 (float): The frequency at which the magnitude of the transfer function is 1.

    Returns
    -------
    TransferFunction: The frequency weight transfer function.

    """
    return ct.TransferFunction([M], [M/w1, 1])


def makeweight_3(M: float, w3 : float) -> ct.TransferFunction:
    """
    Creates a frequency weight transfer function that has:
    - zero DC gain,
    - approaches M >> 1 at high frequencies,
    - and has magnitude 1 at the frequency s=j*w3.

    Parameters
    ----------
    - M (float): The magnitude of the frequency weighting
    - w3: The frequency at which the weighting starts to take effect

    Returns
    -------
    - tf: The transfer function representing the frequency-weighted system
    """

    return ct.TransferFunction([M, 0], [1, M*w3])



def create_weight(wc, low_gain, high_gain):
    """
    Create a weighting function.
    
    Parameters:
    wc : float
        Crossover frequency (where the gain changes).
    low_gain : float
        Gain at low frequencies.
    high_gain : float
        Gain at high frequencies.
    """
    # Magnitude at crossover = geometric mean of low_gain and high_gain
    magnitude_at_wc = np.sqrt(low_gain * high_gain)

    # First order weight transfer function: (low_gain/s) / (1 + s/wc)
    num = [low_gain, 0]
    den = [1/wc, 1]

    return ct.TransferFunction(num, den)

# # Example parameters
# wc = 10  # Crossover frequency at 10 rad/s
# low_gain = 10  # High gain at low frequencies
# high_gain = 1  # Low gain at high frequencies

# weight_tf = create_weight(wc, low_gain, high_gain)

# # Plot the magnitude response
# omega = np.logspace(-1, 2, 500)
# mag, phase, omega = ct.bode(weight_tf, omega, plot=True)

# plt.figure()
# plt.semilogx(omega, 20 * np.log10(mag))
# plt.title('Weighting Function Magnitude Response')
# plt.xlabel('Frequency (rad/s)')
# plt.ylabel('Magnitude (dB)')
# plt.grid(True)
# plt.show()


In [ ]:
import control as ct
import numpy as np
import matplotlib.pyplot as plt

# Define the system dynamics
def inverted_pendulum_update(t, x, u, params={}):
    g = 9.81  # Acceleration due to gravity in m/s^2
    L = 1.0   # Length of the pendulum in meters
    b = 0.1   # Damping coefficient

    theta, theta_dot = x  # Unpack the state variables
    theta += np.pi        # Shift the angle to make the upright position the zero point
    u = u[0]              # Control input (torque)

    # Equation of motion
    theta_ddot = (u - b * theta_dot - g * np.sin(theta)) / L

    return np.array([theta_dot, theta_ddot])

def inverted_pendulum_output(t, x, u, params={}):
    return x[0]  # Output is the angle theta

# Create a NonlinearIOSystem for the pendulum
pendulum_nl = ct.NonlinearIOSystem(
    inverted_pendulum_update,  # Update function
    inverted_pendulum_output,  # Output function
    inputs=('u'),                       # Number of inputs
    outputs=('theta'),                  # Number of outputs
    states=('theta', 'theta_dot'),      # Number of states
    name='pendulum'
)

# Linearize the system around the upright position (theta = 0, theta_dot = 0) with zero input
x0 = [0, 0]  # Operating point (state)
u0 = [0]     # Operating point (input)
t0 = 0       # Time to linearize (usually 0 for time-invariant systems)

# Linearize the nonlinear system at the specified operating point
sys_lin = ct.linearize(pendulum_nl, xeq=x0, ueq=u0, t=t0)

# Convert the linearized state-space model to a transfer function
pendulum_tf = ct.ss2tf(sys_lin)
print("Transfer function of the linearized pendulum:")
print(pendulum_tf)

P_s = pendulum_tf

# Plot the singular value plot for the linearized system
plt.figure()
ct.bode_plot(pendulum_tf)
plt.show()

In [ ]:
# Assume P_s is the transfer function you obtained earlier
poles = ct.pole(pendulum_tf)

# Check if all poles have negative real parts
is_stable = all(p.real < 0 for p in poles)

print("System is stable:" if is_stable else "System is unstable:")
print(poles)

In [ ]:
sys_lin

In [ ]:
A = sys_lin.A

D, V = np.linalg.eig(A)
D

In [ ]:
# Initial conditions: small angle (e.g., 5 degrees converted to radians)
theta_initial = np.deg2rad(1)  # Initial angle 
theta_dot_initial = 0          # Initial angular velocity
x0 = [theta_initial, theta_dot_initial]

# Time vector for the simulation
t = np.linspace(0, 10, 1000)  # Simulate for 10 seconds

# Zero input (no control input)
u = np.zeros_like(t)

# Simulate the response
t, y = ct.input_output_response(pendulum_nl, T=t, U=u, X0=x0)

# Plotting the response
plt.figure()
plt.plot(t, y)
plt.title('Inverted Pendulum Response from Initial Condition')
plt.xlabel('Time (s)')
plt.ylabel('Angle (rad)')
plt.grid(True)
plt.show()

In [ ]:
theta_initial

In [ ]:
import control as ct
import numpy as np
import matplotlib.pyplot as plt

# Assume sys_lin is the linearized state-space model from previous steps
# Linearizing again for clarity in this context
# Define operating points and system parameters as before

# Define weight matrices for LQR
Q = np.diag([100, 1])  # State weight matrix
R = [0.1]              # Input weight matrix

# Compute the LQR controller gain
K, _, _ = ct.lqr(sys_lin.A, sys_lin.B, Q, R)

# Create the closed-loop system (A-BK)
A_cl = sys_lin.A - sys_lin.B @ K
B_cl = sys_lin.B
C_cl = sys_lin.C
D_cl = sys_lin.D

sys_cl = ct.StateSpace(A_cl, B_cl, C_cl, D_cl)

# Initial conditions and simulation parameters
theta_initial = 5 * np.pi / 180  # 5 degrees in radians
theta_dot_initial = 0
x0_cl = [theta_initial, theta_dot_initial]

# Simulate the response
t = np.linspace(0, 10, 1000)
t, y_cl = ct.initial_response(sys_cl, T=t, X0=x0_cl)

# Plotting the controlled response
plt.figure()
plt.plot(t, y_cl)
plt.title('Controlled Response of Inverted Pendulum')
plt.xlabel('Time (s)')
plt.ylabel('Angle (rad)')
plt.grid(True)
plt.show()


In [ ]:
T_s = ct.ss2tf(sys_cl)
print(T_s)

In [ ]:
# Assume P_s is the transfer function you obtained earlier
poles = ct.pole(T_s)

# Check if all poles have negative real parts
is_stable = all(p.real < 0 for p in poles)

print("System is stable:" if is_stable else "System is unstable:")
print(poles)

# LGR LQR

In [ ]:
import numpy as np
import control as ct
import matplotlib.pyplot as plt

# System matrices (assuming you have already a linearized model sys_lin)
A = sys_lin.A
B = sys_lin.B
C = sys_lin.C
D = sys_lin.D

Q = np.diag([100, 1])  # State weight matrix
R = [0.1]              # Input weight matrix

# Design state feedback controller (LQR)
K, S, E = ct.lqr(sys_lin, Q, R)

# Define LQE noise covariances
Qe = np.diag([0.1, 0.1])  # Process noise covariance
Re = np.diag([0.1])  # Measurement noise covariance

# Design observer (LQE)
L, P, E = ct.lqe(sys=sys_lin, QN=Qe, RN=Re)  

# Implement the observer
A_obs = A - L @ C
B_obs = np.hstack([B, L])
C_obs = np.eye(A.shape[0])  # Identity matrix for full state output
D_obs = np.zeros((A.shape[0], B_obs.shape[1]))

observer_ss = ct.StateSpace(A_obs, B_obs, C_obs, D_obs)

# Closed-loop system matrices
A_cl = A - B @ K
B_cl = B.copy()  # For input or external disturbance
C_cl = C.copy()
D_cl = D.copy()

# Combine into one system (assuming direct state measurement for simplicity)
sys_cl = ct.StateSpace(A_cl, B_cl, C_cl, D_cl)

# Simulate the system
t = np.linspace(0, 10, 1000)
x0 = np.array([0.1, 0, 0, 0])  # Initial condition for the states
t, y = ct.initial_response(sys_cl, T=t, X0=x0)

# Plotting the response
plt.figure()
plt.plot(t, y)
plt.title('Response of the Inverted Pendulum with LQR and Luenberger Observer')
plt.xlabel('Time (s)')
plt.ylabel('State response')
plt.grid(True)
plt.show()


# PI Controller

In [ ]:
import control as ct
import numpy as np
import matplotlib.pyplot as plt

# Define the PID parameters
Kp = 50.0  # Proportional gain
Ki = 5.0   # Integral gain
Kd = 10  # Derivative gain

# Create the PID controller transfer function
num = [Kd, Kp, Ki]  # Coefficients of s^2, s, and constant
den = [1, 0]        # Coefficient for s in the denominator makes it proper
C_s = ct.TransferFunction(num, den)

print("PID Controller Transfer Function:")
print(C_s)

print("Plant Transfer Function:")
print(pendulum_tf)

# Assume sys_lin is your plant model obtained by linearizing the nonlinear system
# Combine the controller and the plant in a feedback loop
system_open_loop = C_s * pendulum_tf
print("Open-loop Transfer Function:")
print(system_open_loop)

system_closed_loop = ct.feedback(system_open_loop, 1)
print("Closed-loop Transfer Function:")
print(system_closed_loop)

# Simulate the response to an initial condition
t = np.linspace(0, 10, 1000)
initial_condition = [np.deg2rad(5), 0, 0]  # Starting with a small angle deviation, the third argument is the initial error
t, y = ct.initial_response(system_closed_loop, T=t, X0=initial_condition)

# Plotting the response
plt.figure()
plt.plot(t, y)
plt.title('Response of the Inverted Pendulum with PID Controller')
plt.xlabel('Time (s)')
plt.ylabel('Angle (rad)')
plt.grid(True)
plt.show()


In [ ]:
print(system_closed_loop)

In [ ]:
# Assume P_s is the transfer function you obtained earlier
poles = ct.pole(system_closed_loop)

# Check if all poles have negative real parts
is_stable = all(p.real < 0 for p in poles)

print("System is stable:" if is_stable else "System is unstable:")
print(poles)